# 词向量的应用

在Word2Vec出现之前， 我们通常会使用onehot-encoding 的方式对词语进行向量化表示(向量中有一位为1， 其余为0)， 这种方式存在两个重要对缺陷：  
- onehot词向量大小会受Vocabulary（词典）大小的影响， 通常使用onehot-encoding的词向量维度会等同于词典大小。
- onehot词向量无法直接体现词义。     


通过使用Word2Vec算法， 可以将词语表示为维度较低的N维向量， 向量本身包含对词义的编码（训练方式本身就是基于特定文本窗口大小的语境进行计算的）， 所以可以完成类似词语距离计算的应用。

## 准备工作

首先需要结合语料， 进行词向量的训练， 生成词向量模型。本项目提供了两种词向量的训练方式：  
- 使用gensim提供的模型（包括word2vec以及fasttext）， 进行模型训练(代码位于WordEmbedding/gensim_w2v/下, 训练方式详见目录下的README.md文件)
- 使用pytorch从零开始构建模型进行词向量训练（comming soom...）

## 词向量应用

词向量最核心的应用就是词语间的距离计算， 下面我们来看一下具体的应用：

In [3]:
# model 使用的jieba分词以及size=500的模型参数
model_loc = '/Users/jiangchaodi/PycharmProjects/NLPRoadMap/WordEmbedding/genism_w2v/model/zh_wiki.model'

In [4]:
from gensim.models import Word2Vec

In [13]:
# 加载模型
model = Word2Vec.load(model_loc)

#### 获取词向量

In [15]:
足球 = model.wv['足球']
足球

array([ 1.0544275 ,  0.23436378, -0.7143615 ,  1.4122411 , -0.51715213,
       -0.46913198, -0.8298384 , -0.48959342, -0.6144801 ,  1.1951568 ,
        0.8780931 , -0.5382308 , -1.2539213 , -0.06205823, -1.5639577 ,
        1.3295213 ,  0.2351637 , -1.4662336 ,  0.7610299 ,  0.29038516,
       -0.8379624 ,  0.19686222,  0.67414105,  0.63005906, -1.0664047 ,
        0.4669381 , -1.5620009 ,  0.46362957, -0.55529344, -2.2557976 ,
       -1.9426118 ,  0.3375667 , -1.313447  , -1.28838   ,  0.37956038,
        0.70486546,  1.3499844 ,  1.9196879 , -1.0190893 , -1.9122719 ,
        0.47655565,  1.0093675 , -1.5483143 ,  0.63550276, -1.8836249 ,
       -0.2343712 , -0.8515781 , -1.5061207 , -1.3749253 , -0.7814426 ,
       -0.83903986, -2.3784926 ,  1.7621362 ,  1.1979053 , -0.74346495,
        0.0269281 , -0.0805449 ,  0.10933974, -1.1726516 , -1.7880434 ,
       -0.31660643,  2.0721292 ,  0.5256784 , -1.8122436 , -1.351268  ,
       -0.477671  ,  1.3142369 ,  0.33640158,  1.0081137 , -1.41

In [16]:
足球.shape

(500,)

#### 相似度计算

In [21]:
# 计算‘上海’和‘上海市’之间的相似度
model.wv.similarity('上海', '上海市')

0.6461843

#### 最相似词语

本质上就是计算语料库中所有词语和目标词语的相似度， 并返回距离相似度最高的词语

In [10]:
model.wv.most_similar('足球', topn=5)

[('国际足球', 0.5315186977386475),
 ('足球运动', 0.5238089561462402),
 ('篮球', 0.5211929082870483),
 ('足球队', 0.49278587102890015),
 ('排球', 0.486592561006546)]

#### 类比

在已知道A, B, C的情况下求D， 解决 A->B:C-D的类比问题

In [24]:
model.wv.most_similar(positive=['女人', '国王'], negative=['男人'], topn=1)

[('王后', 0.492384135723114)]

## 基于词向量进行检索

假设我们的知识库以键值的形式存储在数据库， 其中键是以词语的形式存储的。那么当用户输入一个关键词， 就能通过计算用户的关键词与资源库（知识库）中关键词的相似度， 来获取用户想要的信息。如果用户输入的是一个句子， 则可以进行分词， 或者使用如下方法进行关键信息(基于语法分析， 提取诸如主语之类的重要信息)的提取：

In [34]:
from pyhanlp import HanLP
print(HanLP.extractKeyword('你的故乡在哪里', 1))

[故乡]


同时， 如果知识库中有[故乡]的键存在， 那么与[故乡]相似度最高的键对应的答案将被返回（比如[家乡]）

但是，如果知识库中的问题是一个句子的形式存在的，比如对于‘驾驶证过期之后， 如何补办？’ 这样的问题， 虽然依然可以提取关键信息， 比如我们选择对这句话提取三个关键词：

In [39]:
print(HanLP.extractKeyword('驾驶证过期之后， 如何补办？',3))

[过期, 之后, 补办]


这里最重要的‘驾驶证’没有被提取出来。同时在知识库中， 只用一个或若干个关键词来对应知识，来很好地描述及概括一个知识， 同时又不与其他的问题产生冲突。

这种情况下， 采取对文档的编码， 比如Doc2Vec（comming soon）或者[利用bert对语句进行编码](https://nbviewer.jupyter.org/github/superjcd/NLPRoadMap/blob/master/NoteBooks/%E8%AF%AD%E4%B9%89%E7%9B%B8%E4%BC%BC%E5%BA%A6%E8%AE%A1%E7%AE%97_BERT.ipynb)就显的很重要。